In [ ]:
pip install -U -q bitsandbytes datasets transformers peft "transformers[sentencepiece]" 

In [ ]:
# --- Block 1: Imports and Configuration ---

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import warnings 
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Transformers and PEFT (for QLoRA)
import transformers
from transformers import (
    AutoModel, 
    AutoTokenizer, 
    BitsAndBytesConfig,
    DataCollatorWithPadding
)
from peft import (
    LoraConfig, 
    get_peft_model, 
    PeftModel
)

# bitsandbytes for 8-bit optimizer
import bitsandbytes as bnb

# Suppress warnings
warnings.filterwarnings("ignore")
torch.cuda.is_available()

In [ ]:
#
train = pd.read_csv("/teamspace/studios/this_studio/data/train.csv")
test = pd.read_csv("/teamspace/studios/this_studio/data/test.csv")
sub = pd.read_csv("/teamspace/studios/this_studio/data/sample_submission.csv")

#
train = train[['id'	,'prompt',	'response_a'	,'response_b',	'winner_model_a'	,'winner_model_b',	'winner_tie']] 

In [ ]:
# remove [''] and creating class 
def preprocess(df, istrain):
    """
    Cleans text columns and creates a 'class' label from winner columns.
    
    Args:
        df (pd.DataFrame): The raw DataFrame with columns like 'prompt', 
                           'winner_model_a', etc.
                           
    Returns:
        pd.DataFrame: A new DataFrame with cleaned text columns and a new 'class' column.
    """
    
    df_processed = df.copy()

    text_cols = ['prompt', 'response_a', 'response_b']
    for col in text_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].str.strip('[]"\'') 

    
    if(istrain):
        
        conditions = [
            (df_processed['winner_model_a'] == 1),
            (df_processed['winner_model_b'] == 1),
            (df_processed['winner_tie'] == 1)
        ]
        
        choices = [0, 1, 2]
        df_processed['class'] = np.select(conditions, choices)
    
    return df_processed